In [ ]:
"""
Working with observations.

This notebook show you how to fetch observations from the database.

The records loaded from spreadsheet rows or returned from the eBird API
are saved across five different models: Checklist, Location, Observer,
Observation and Species, which saves a lot of space. However, that means
you have to join the records from each table together when processing
Checklists or Observations. That's easy to do with SQLAlchemy, but eBird
notebooks also provides a helper class, ObservationQuery, which implements
the most common queries when fetching Observations.
"""

import datetime as dt

from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session

from ebird.notebooks import settings
from ebird.notebooks.loaders import APILoader
from ebird.notebooks.models import Checklist, Location
from ebird.notebooks.queries import ObservationQuery  # noqa : E402

api_key = settings.API_KEY
db_url = settings.API_DB_URL
engine = create_engine(db_url)

# Convenience function for displaying the checklists fetched.

# Each row contains three objects, the Checklist, the checklist's Location,
# and the Observer who submitted the checklist. To access each attribute, use
# the model name and attribute name. See ebird.notebooks.models for the list
# of models and attributes. Checklists are returned, sorted by checklist date
# and time, oldest first.
#
# For brevity and performance, the number of checklists shown is limited to 10.
def show_rows(rows):
    for idx, row in enumerate(rows):
        print(row.Observation.count, row.Species.common_name, row.Checklist.date, row.Checklist.time, row.Location.name)
        if idx == 10: break

In [ ]:
# Load the observations from the eBird API.
loader = APILoader(api_key, db_url)
today = dt.date.today()

for region in settings.API_REGIONS:
    loader.load(region, today)

In [ ]:
# Get the Location from the most recent Checklist to get some values
# for the example code in the following cells.

with Session(engine) as session:
    row = session.execute(select(Checklist, Location).join(Location).order_by(Checklist.identifier.desc())).first()

country = row.Location.country
country_code = row.Location.country_code
state = row.Location.state
state_code = row.Location.state_code
county = row.Location.county
county_code = row.Location.county_code

print("Country: ", country)
print("Country code: ", country_code)
print("State: ", state)
print("State code: ", state_code)
print("County: ", county)
print("County code: ", county_code)

In [ ]:
# Fetch every checklist.
with Session(engine) as session:
    rows = ObservationQuery(session).fetch()
    show_rows(rows)

In [ ]:
# Count the number of checklists.
with Session(engine) as session:
    year = today.year
    count = ObservationQuery(session).for_year(year).count()
    print("Checklists submitted in %d: " % year, count)

In [ ]:
# Fetch the checklists for a given country.

In [ ]:
# Using the name of the country.
with Session(engine) as session:
    rows = ObservationQuery(session).for_country(country).fetch()
    show_rows(rows)

In [ ]:
# Or using the country code.
with Session(engine) as session:
    rows = ObservationQuery(session).for_country(country_code).fetch()
    show_rows(rows)

In [ ]:
# Fetch the checklists for a given state (subnational1 area).

In [ ]:
# Using the name of the state.
with Session(engine) as session:
    rows = ObservationQuery(session).for_state(state).fetch()
    show_rows(rows)

In [ ]:
# Or using the code for the state.
with Session(engine) as session:
    rows = ObservationQuery(session).for_state(state_code).fetch()
    show_rows(rows)

In [ ]:
# Fetch the checklists for a given region.
# This is just an alias for fetching checklists by state.

In [ ]:
# Using the name of the region
with Session(engine) as session:
    rows = ObservationQuery(session).for_region(state).fetch()
    show_rows(rows)

In [ ]:
# Or using the code for the region.
with Session(engine) as session:
    rows = ObservationQuery(session).for_region(state_code).fetch()
    show_rows(rows)

In [ ]:
# Fetch the checklists for a given county.

In [ ]:
# Using the name of the county
with Session(engine) as session:
    rows = ObservationQuery(session).for_county(county).fetch()
    show_rows(rows)

In [ ]:
# Or using the code for the county.
with Session(engine) as session:
    rows = ObservationQuery(session).for_county(county_code).fetch()
    show_rows(rows)

In [ ]:
# Fetch the checklists for a given year.
with Session(engine) as session:
    year = today.year
    rows = ObservationQuery(session).for_year(year).fetch()
    show_rows(rows)

In [ ]:
# Fetch the checklists for a given month.
with Session(engine) as session:
    rows = ObservationQuery(session).for_month(today.year, today.month).fetch()
    show_rows(rows)

In [ ]:
# Fetch the checklists for a given day.
with Session(engine) as session:
    rows = ObservationQuery(session).for_day(today.year, today.month, today.day).fetch()
    show_rows(rows)

In [ ]:
# Fetch the checklists for a given date.
with Session(engine) as session:
    rows = ObservationQuery(session).for_date(today).fetch()
    show_rows(rows)